In [ ]:
from elvia_stats.metervalueapi import get_metervalues
import pandas as pd
import altair as alt

In [ ]:
json = get_metervalues(2021)

In [ ]:
df = pd.json_normalize(
    json["meteringpoints"][0], record_path=["metervalue", "timeSeries"]
)

## Handle time
Time is read as string. We should convert it to a datetime. 

Using pd.to_datetime() did not return a timezone aware column type. Seems to be a problem with summer/winter time hence we convert to UTC and then to "Europe/Oslo" timezone instead of +01 or +02

In [ ]:
df["startTimeStr"] = df["startTime"]
df["startTime"] = pd.to_datetime(df["startTime"], utc=True).dt.tz_convert("Europe/Oslo")
df["endTime"] = pd.to_datetime(df["endTime"], utc=True).dt.tz_convert("Europe/Oslo")
df.describe()

In [ ]:
df.head()

In [ ]:
alt.Chart(df[["startTime", "value"]].tail(24 * 30)).mark_bar().encode(
    x="startTime:T", y="value", tooltip=["startTime:T", "value"]
)

In [ ]:
df["hour"] = df["startTime"].dt.hour
df["weekday"] = df["startTime"].dt.weekday
df["date"] = df["startTime"].dt.date.astype(str)
df["month"] = df["startTime"].dt.month
df.dtypes

In [ ]:
alt.Chart(df[["hour", "value"]].tail(24 * 30)).mark_boxplot().encode(
    x=alt.X("hour"), y=alt.Y("value")
)

In [ ]:
df.sort_values("value", ascending=False).head()

In [ ]:
df_max_per_day = df[["startTime", "value"]].groupby(by=df["startTime"].dt.date).max()
df_max_per_day

In [ ]:
df_max_per_day.plot(y="value")

In [ ]:
df_max_per_week = (
    df[["startTime", "value"]].groupby(by=df["startTime"].dt.isocalendar().week).max()
)
df_max_per_week

In [ ]:
df_max_per_week.plot(y="value")

In [ ]:
df_max_per_month = df[["startTime", "value"]].groupby(by=df["startTime"].dt.month).max()
df_max_per_month

In [ ]:
df_max_per_month.plot(y="value")

In [ ]:
df.loc[
    df[["startTime", "value"]].groupby(by=df["startTime"].dt.month).idxmax()["value"]
]

# New nettleie models
[Example models used](https://kommunikasjon.ntb.no/pressemelding/elvia-ruller-ut-pilot-for-ny-nettleie?publisherId=17847555&releaseId=17896111)

| Nettleie                       Dag&Natt       |         Sommer |        Vinter |
| -------------------------------------------   | -------------- | ------------- |
| Energiledd hverdag, kl. 06-22                 | 27,65 øre/kWh  | 71,70 øre/kWh |
| Energiledd natt og helg*                      | 25,15 øre/kWh  | 28,90 øre/kWh |
| Fastledd Innlandet                            | 200 kr/mnd     |    200 kr/mnd |  
| Fastledd Oslo/Viken                           | 115 kr/mnd     |    115 kr/mnd |

* Helg er hele lørdag, søndag og offentlige fridager. Sommer er fra april til og med oktober. Vinter er fra november til og med mars.

In [ ]:
df_line_rental_prices = pd.DataFrame(
    [
        ["summer_day", 0.2765],
        ["summer_night", 0.2515],
        ["winter_day", 0.7170],
        ["winter_night", 0.2890],
    ],
    columns=["line_rental_period", "line_rental_cost_nok_per_kwh"],
)
df_line_rental_prices

In [ ]:
df

In [ ]:
df1 = pd.DataFrame(
    {"startTime": ["a", "b", "c", "e"], "hour": [4, 5, 6, 7], "month": [7, 11, 11, 11]}
)
df1

In [ ]:
def tag_line_rental_nigh_day(df1):
    df1["line_rental_time_of_day"] = "day"
    df1.loc[
        (df1["hour"] < 6) | (df1["hour"] >= 22), "line_rental_time_of_day"
    ] = "night"

    df1["line_rental_season"] = "summer"
    df1.loc[(df1["month"] < 4) | (df1["month"] > 10), "line_rental_season"] = "winter"
    df1["line_rental_period"] = (
        df1["line_rental_season"] + "_" + df1["line_rental_time_of_day"]
    )
    return df1


df1 = tag_line_rental_nigh_day(df1)
df1

df1

In [ ]:
def add_prices(df1):
    return df1.merge(
        df_line_rental_prices,
        left_on="line_rental_period",
        right_on="line_rental_period",
    ).sort_values("startTime")


add_prices(df1)

In [ ]:
add_prices(tag_line_rental_nigh_day(df))